In [34]:
import numpy as np
import cv2

def hFilter(im):
  return np.delete(np.abs(im[:,1:]-im[:,0:-1]),0,0)
def vFilter(im):
  return np.delete(np.abs(im[1:,:]-im[0:-1,:]),0,1)

def train(path_to_images = 'data/training/images', csv_file = 'data/training/steering_angles.csv'):
  data=np.genfromtxt(csv_file, delimiter = ',')
  ims=data[:,0]
  T=data[:,1]
  im_full = cv2.imread(path_to_images + '/' + str(int(0)).zfill(4) + '.jpg')
  im_full = np.mean(im_full[532:,:,:]/255,axis=2)
  im_full[im_full<.9]=0
  im_full=np.sqrt(hFilter(im_full)**2+vFilter(im_full)**2)
  NN=NeuralNet(im_full.flatten())
  X=[]
  for i in ims:
    #print(i%100)
    if int(i+1)%375!=0:
      im_full = cv2.imread(path_to_images + '/' + str(int(i)).zfill(4) + '.jpg')
      im_full = np.mean(im_full[532:,:,:]/255,axis=2)
      im_full[im_full<.9]=0
      im_full=np.sqrt(np.delete(np.abs(im_full[:,1:]-im_full[:,0:-1]),0,0)**2+np.delete(np.abs(im_full[1:,:]-im_full[0:-1,:]),0,1)**2)
      X.append(im_full.flatten())
    else:
      NN.forward(X,np.reshape(T[int(i-374):int(i)],(-1,1)),int(i))
      X=[]
  return NN



def predict(NN, image_file):
      im_full = cv2.imread(image_file)
      im_full = np.mean(im_full[532:,:,:]/255,axis=2)
      im_full[im_full<.9]=0
      imx=hFilter(im_full)
      imy=vFilter(im_full)
      im_full=np.sqrt(imx**2+imy**2)
      X=im_full.flatten()
      X1=NN.addOnes([X])
      Z = np.tanh(X1 @ NN.V)
      Z1 = NN.addOnes(Z)
      Y = Z1 @ NN.W
      #print(Y,Z)
      return Y[0,0]

class NeuralNet(object):
  def __init__(self,X):        
        '''
        Neural Network Class, you may need to make some modifications here!
        '''
        self.inputLayerSize = X.size
        self.outputLayerSize = 1
        self.hiddenLayerSize = 12
        self.error=0
        self.rh=.2/ (499)
        self.ro=18/ (499)
        #self.V = np.random.randn(self.inputLayerSize+1,self.hiddenLayerSize)
        #self.W = np.random.randn(self.hiddenLayerSize+1,self.outputLayerSize)
        self.V = 0.1*1*(np.random.uniform(size=(self.inputLayerSize+1,self.hiddenLayerSize))-0.5)
        self.W = 0.1*1*(np.random.uniform(size=(1+self.hiddenLayerSize,self.outputLayerSize))-0.5)
  
  def addOnes(self,A):
    return np.insert(A, 0, 1, axis=1)

  def forward(self,X,t,i):
    X=self.addOnes(X)
    for i in range(18):    
      #print(X1.shape,self.V.shape,Z1.shape,self.W.shape,Y.shape,np.array(t).shape)
      self.error = t-self.addOnes(np.tanh(X @ self.V)) @ self.W
      self.backwards(X,self.addOnes(np.tanh(X @ self.V)),self.addOnes(np.tanh(X @ self.V)) @ self.W)
  
  def backwards(self,X,Z,Y):
    self.V+=self.rh*(1/Y.shape[0])*(1/Y.shape[1])*X.T@np.multiply((self.error*self.W[1:].T),(1-Z[:,1:]**2))
    self.W+=self.ro*(1/Y.shape[0])*(1/Y.shape[1])*Z.T@self.error

In [35]:
import numpy as np
import cv2
from tqdm import tqdm
import time
import signal


def timeout_handler(num, stack):
    raise Exception("TIMEOUT")


def evaluate(path_to_training_images='C:\\Users\\zhami\\Downloads\\training\\images',
                    training_csv_file='C:\\Users\\zhami\\Downloads\\training\\steering_angles.csv',
                    path_to_testing_images='C:\\Users\\zhami\\Downloads\\training\\images',
                    testing_csv_file='C:\\Users\\zhami\\Downloads\\training\\steering_angles.csv',
                    time_limit=600):
    '''
    Evaluate Student Submission for autonomous driving challenge.
    Train and test studen't neural network implementation.
    Training time is limited to time_limit seconds, if your code takes
    longer than this, it will be terminated and no score will be recorded.
    '''

    # Setup timout handler - I think this will only work on unix based systems:


    try:
        print("Training your network, start time: %s" % time.strftime("%H:%M:%S"))
        NN = train(path_to_images=path_to_training_images,
                   csv_file=training_csv_file)

    finally:
        print("Ending Time: %s" % time.strftime("%H:%M:%S"))

    print('Training Complete! \n')

    print('Measuring performance...')
    ## Measure Performance:
    data = np.genfromtxt(testing_csv_file, delimiter=',')
    frame_nums = data[:, 0]
    steering_angles = data[:, 1]

    predicted_angles = []
    for frame_num in tqdm(frame_nums):
        im_path = path_to_testing_images + '/' + str(int(frame_num)).zfill(4) + '.jpg'
        predicted_angles.append(predict(NN, im_path))

    RMSE = np.sqrt(np.mean((np.array(predicted_angles) - steering_angles) ** 2))
    RMSE = round(RMSE, 3)
    print('Test Set RMSE = ' + str(RMSE) + ' degrees.')

    return RMSE


def calculate_score(RMSE):
    score = 0
    if RMSE <= 12.5:
        score = 10
    elif RMSE <= 15:
        score = 9
    elif RMSE <= 20:
        score = 8
    elif RMSE <= 25:
        score = 7
    elif RMSE <= 30:
        score = 6
    elif RMSE <= 40:
        score = 5
    else:
        score = 4
    return score


if __name__ == '__main__':
    program_start = time.time()
    RMSE = evaluate(path_to_training_images='C:\\Users\\zhami\\Downloads\\training\\images',
                    training_csv_file='C:\\Users\\zhami\\Downloads\\training\\steering_angles.csv',
                    path_to_testing_images='C:\\Users\\zhami\\Downloads\\training\\images',
                    testing_csv_file='C:\\Users\\zhami\\Downloads\\training\\steering_angles.csv',
                    time_limit=600)

    score = calculate_score(RMSE)
    program_end = time.time()
    total_time = round(program_end - program_start, 2)

    print("Execution time (seconds) = ", total_time)
    print("Score = ", score)

Training your network, start time: 23:39:13


  0%|                                                                                         | 0/1500 [00:00<?, ?it/s]

Ending Time: 23:42:06
Training Complete! 

Measuring performance...


100%|██████████████████████████████████████████████████████████████████████████████| 1500/1500 [01:55<00:00, 12.99it/s]

Test Set RMSE = 12.505 degrees.
Execution time (seconds) =  288.5
Score =  9
